# Multi-Model AI Assitant with Context History!

Utilize the power of OpenAI's ChatGPT and Google's Gemini at the same time.

In [74]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai as genai
import gradio as gr

In [75]:
# set up environment

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
gemini_key = os.getenv('GEMINI_API_KEY')


In [76]:
openai_system_message = "You are a helpful assistant called OpenAI or ChatGPT or GPT. The user is talking to you and another assistant called Gemini at the same time, \
using a single text input. So if the user provides specific instructions for Gemini or calls out Gemini, you can ignore those."
gemini_system_message = "You are a helpful assistant called Gemini or Google or Google's LLM. The user is talking to you and another assistant called openai at the same time, \
using a single text input. So if the user provides specific instructions for openai or calls out OpenAI, you can ignore those."
openai_history = []
gemini_history = []

In [77]:
def chat(message, openai_history, gemini_history):
    print("part1", openai_history, gemini_history)
    
    if openai_history is None:
        openai_history = []  
    if gemini_history is None:
        gemini_history = []
        
    openai_messages = [{"role": "system", "content": openai_system_message}] + openai_history + [{"role": "user", "content": message}]
    #gemini_messages = [{"role": "user", "content": message}]

    gemini_history_string = ""
    for entry in gemini_history:
        if entry["role"] == "user":
            gemini_history_string += f"User: {entry['content']}\n"
        else:
            gemini_history_string += f"Assistant: {entry['content']}\n"
    
    gemini_prompt = f"{system_message}\n{gemini_history_string}User: {message}\nAssistant:"
    
    """
    gemini_history = ""
    for entry in openai_history:
        if entry["role"] == "user":
            gemini_history += f"User: {entry['content']}\n"
        else:
            gemini_history += f"Assistant: {entry['content']}\n"
    
    gemini_prompt = f"{system_message}\n{gemini_history}User: {message}\nAssistant:"
    """
    
    """
    gemini_history = "\n".join(
        [f"User: {entry['content']}" if entry["role"] == "user" else f"Assistant: {entry['content']}" for entry in openai_history]
    )
    gemini_prompt = f"{system_message}\n{gemini_history}\nUser: {message}\nAssistant:"
    """
    
    genai.configure()
    gemini = genai.GenerativeModel(
        model_name = "gemini-1.5-flash",
        system_instruction = gemini_system_message,
        )
    
    openai = OpenAI()
    MODEL_GPT = 'gpt-4o-mini'

    openai_responses = openai.chat.completions.create(model = MODEL_GPT, messages = openai_messages).choices[0].message.content
    gemini_responses = gemini.generate_content(gemini_prompt).text
    
    openai_history += [{"role": "user", "content": message}, {"role": "assistant", "content": openai_responses}]
    gemini_history += [{"role": "user", "content": message}, {"role": "assistant", "content": gemini_responses}]
    
    print("Part 2 - OpenAI History:", openai_history)
    print("Part 2 - Gemini History:", gemini_history)
    
    return openai_responses, gemini_responses, openai_history, gemini_history


In [78]:
# Gradio 

with gr.Blocks() as ui:
    
    gr.Markdown("## Multi-Model AI Assitant")
    
    with gr.Row():
        message = gr.Textbox(label="Hiii, I am your assistant")

    with gr.Row():
        fetch = gr.Button("Do the magic!")

    with gr.Row():
        with gr.Column():
            gr.Markdown("### OpenAI Response:")
            openai_responses = gr.Markdown()

        with gr.Column():
            gr.Markdown("### Gemini Response:")
            gemini_responses = gr.Markdown()

    openai_history = gr.State([])
    gemini_history = gr.State([])

    """
    def add_history(message, history):
        history += [{"role":"user", "content":message}]
        return "", history
    """
    #fetch.click(chat, inputs=[message, openai_history], outputs=[openai_responses, gemini_responses, openai_history])
    fetch.click(chat, inputs=[message, openai_history, gemini_history], outputs=[openai_responses, gemini_responses, openai_history, gemini_history])

    print(openai_history, gemini_history)
    
ui.launch()

<gradio.components.state.State object at 0x0000020A46BE5B90> <gradio.components.state.State object at 0x0000020A46B31610>
* Running on local URL:  http://127.0.0.1:7883

To create a public link, set `share=True` in `launch()`.


part1 [] []
Part 2 - OpenAI History: [{'role': 'user', 'content': 'Hi I am Aadesh'}, {'role': 'assistant', 'content': 'Hello Aadesh! How can I assist you today?'}]
Part 2 - Gemini History: [{'role': 'user', 'content': 'Hi I am Aadesh'}, {'role': 'assistant', 'content': "Hi Aadesh, it's nice to meet you! How can I help you today?\n"}]
part1 [{'role': 'user', 'content': 'Hi I am Aadesh'}, {'role': 'assistant', 'content': 'Hello Aadesh! How can I assist you today?'}] [{'role': 'user', 'content': 'Hi I am Aadesh'}, {'role': 'assistant', 'content': "Hi Aadesh, it's nice to meet you! How can I help you today?\n"}]
Part 2 - OpenAI History: [{'role': 'user', 'content': 'Hi I am Aadesh'}, {'role': 'assistant', 'content': 'Hello Aadesh! How can I assist you today?'}, {'role': 'user', 'content': 'I like to play table tennis'}, {'role': 'assistant', 'content': "That's great! Table tennis is a fun and fast-paced sport. How long have you been playing? Do you have a favorite player or style of play?"